In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
#cargar el documento de datos
dataset = pd.read_csv('insurance.csv')


In [2]:
#cargar el documento de datos
dataset = pd.read_csv('insurance.csv')
print(dataset)

      age     sex     bmi  children smoker     region      charges
0      19  female  27.900         0    yes  southwest  16884.92400
1      18    male  33.770         1     no  southeast   1725.55230
2      28    male  33.000         3     no  southeast   4449.46200
3      33    male  22.705         0     no  northwest  21984.47061
4      32    male  28.880         0     no  northwest   3866.85520
...   ...     ...     ...       ...    ...        ...          ...
1333   50    male  30.970         3     no  northwest  10600.54830
1334   18  female  31.920         0     no  northeast   2205.98080
1335   18  female  36.850         0     no  southeast   1629.83350
1336   21  female  25.800         0     no  southwest   2007.94500
1337   61  female  29.070         0    yes  northwest  29141.36030

[1338 rows x 7 columns]


In [4]:
#seprarar caracteristicas (x) y variable objetivo (y)
X = dataset.iloc[:, :-1].values
Y = dataset.iloc[:, -1].values


In [15]:
#verificacion de valores nulos por columna
print(dataset.isnull().sum())
#imputacion de los valores faltantes en numericos
#imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
#imputer.fit(X[:,[0,2,3]]) #columnas numericas
#X[:, [0, 2, 3]] = imputer.transform(X[:, [0, 2, 3]])
#print(X)



age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64


In [19]:
df_numeric = pd.DataFrame(X[:, [0, 2, 3]], columns=['age', 'bmi', 'children'])
for column in df_numeric.columns:
  Q1 = df_numeric[column].quantile(0.25)
  Q3 = df_numeric[column].quantile(0.75)
  IQR = Q3 - Q1
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR
  #capear los valores fuera de los limites
  df_numeric[column] = np.clip(df_numeric[column], lower_bound, upper_bound)
  #actualizar los valores de x sin los outliers
X[:, [0, 2, 3]] = df_numeric.values
print(df_numeric)

      age  bmi children
0     1.0  1.0      0.0
1     0.0  1.0      0.0
2     0.0  1.0      0.0
3     0.0  1.0      0.0
4     0.0  1.0      0.0
...   ...  ...      ...
1333  0.0  1.0      0.0
1334  1.0  1.0      0.0
1335  1.0  1.0      0.0
1336  1.0  1.0      0.0
1337  1.0  1.0      0.0

[1338 rows x 3 columns]


In [23]:
#codificacion para las columnas categoricas
ct =ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [5])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
le_sex = LabelEncoder()
X[:, 1] = le_sex.fit_transform(X[:, 1])   # columna sex
le_smoker = LabelEncoder()
X[:, 4] = le_smoker.fit_transform(X[:, 4])  # columna smoker
print(X)


[[1.0 0 1.0 ... 19.0 27.9 0.0]
 [0.0 1 1.0 ... 18.0 33.77 1.0]
 [0.0 1 1.0 ... 28.0 33.0 3.0]
 ...
 [1.0 0 1.0 ... 18.0 36.85 0.0]
 [1.0 0 1.0 ... 21.0 25.8 0.0]
 [1.0 0 1.0 ... 61.0 29.07 0.0]]


In [26]:
#division en conjuntos de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [29]:
#8. escalado de caracteristicas numericas
sc = StandardScaler()
X_train[:, 9:] = sc.fit_transform(X_train[:, 9:])  # numericas estan desde la columna 9
X_test[:, 9:] = sc.transform(X_test[:, 9:])
print("\nDimensiones finales:")
print("X_train:", X_train.shape, "X_test:", X_test.shape)


Dimensiones finales:
X_train: (1070, 13) X_test: (268, 13)
